In [1]:
!pip install textacy  spacy pandas unidecode vaderSentiment  openpyxl -q
!python -m spacy download pt_core_news_sm -q

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [2]:
#Importar libraries
import pandas as pd
from unidecode import unidecode
import spacy
nlp = spacy.load("pt_core_news_sm")
from textacy.extract.basics import ngrams

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
def process_dicionario(df, column= 'PALAVRA'):
    df[column] = df[column].str.strip() #remove espaços em branco
    df[column] = df[column].str.lower() #transforma em minúsculo
    df[column] = df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') #remove acentos
    return df

In [4]:
#importar dicionarios
dicionario_ambiental = pd.read_excel('V2. DICIONÁRIO COMPILADOS ING.POR.xlsx',sheet_name='AMBIENTAL')
dicionario_social = pd.read_excel('V2. DICIONÁRIO COMPILADOS ING.POR.xlsx',sheet_name='SOCIAL')
dicionario_governanca = pd.read_excel('V2. DICIONÁRIO COMPILADOS ING.POR.xlsx',sheet_name='GOVERNANÇA')

In [5]:
#Processa o dicionario
dicionario_ambiental = process_dicionario(dicionario_ambiental)
dicionario_social = process_dicionario(dicionario_social)
dicionario_governanca = process_dicionario(dicionario_governanca)

In [6]:
#Transforma em listas
dicionario_ambiental = list(dicionario_ambiental['PALAVRA'])
dicionario_social = list(dicionario_social['PALAVRA'])
dicionario_governanca = list(dicionario_governanca['PALAVRA'])

In [7]:
#importa os dados primarios
dados_primarios =   pd.read_excel('DadosLuisaFinalV4.xlsx')

In [8]:
dados_primarios['Nome do Hotel']

0                      ACasaRosa Hostel
1                 Apoema Suites e Flats
2                           Apple House
3                     Bahia Inn Pousada
4                            Casa Clara
                     ...               
191              VILLA CANZIANI &DONATO
192              VILLA ECOPORAN ITACARE
193    Vitória Hotel  Express Dom Pedro
194              Windsor Brasilia Hotel
195        Windsor Plaza Brasilia Hotel
Name: Nome do Hotel, Length: 196, dtype: object

In [9]:
#Relative Path
path = './COMENTS TRIP. HOTÉIS. NOMES CORRETOS. SEM COMENT REPETIDO/'

quantidade_comentarios = dict()

counter_hotel =  0
total_hotel = dados_primarios.shape[0]

for hotel in dados_primarios['Nome do Hotel']:
    counter_hotel += 1
    print(f'Processando hotel {counter_hotel} de {total_hotel} ',hotel.strip())
    hotel_key = hotel
    hotel = hotel.replace('\'', '_')
    hotel = hotel.replace('|', '')
    filename = path + hotel.strip() + '.xlsx'
    dados_comentarios = pd.read_excel(filename)

    #Processa os comentários
    comentarios = dados_comentarios['AVALIAÇÃO']

    counter_governanca = 0
    counter_ambiental = 0
    counter_social = 0
    couter_palavras = 0
    counter_ngram = 0
    ambiental_postivo = 0
    ambiental_negativo = 0
    social_postivo = 0
    social_negativo = 0
    governanca_postivo = 0
    governanca_negativo = 0

    try:
        for comentario in comentarios:

            if len(comentario) > 1:
                
                comentario = comentario.strip()
                comentario = comentario.lower()
                comentario = comentario.replace('\W', '')
                comentario = unidecode(comentario)
                

                #tokeniza o comentario
                corpus = nlp(comentario, disable=['ner', 'entity_linker', 'textcat', 'entitry_ruler'])

                for palavra in corpus:
                    if palavra.is_alpha:
                        couter_palavras += 1


                #n_gramas
                comentarios_ngram = ngrams(corpus, n=range(1,10), filter_stops=False, filter_punct=True, filter_nums=False, include_pos=False, min_freq=1)
                comentarios_ngram = [str(ngram.text).lower() for ngram in comentarios_ngram]

                for ngram in comentarios_ngram:
                    if ngram in dicionario_governanca:
                        counter_governanca += 1

                        sentiment = analyzer.polarity_scores(comentario)
                        if sentiment['compound'] > 0:
                            governanca_postivo += 1
                        else:
                            governanca_negativo += 1


                    if ngram in dicionario_ambiental:
                        counter_ambiental += 1

                        sentiment = analyzer.polarity_scores(comentario)
                        if sentiment['compound'] > 0:
                            ambiental_postivo += 1
                        else:
                            ambiental_negativo += 1

                    if ngram in dicionario_social:
                        counter_social += 1

                        sentiment = analyzer.polarity_scores(comentario)
                        if sentiment['compound'] > 0:
                            social_postivo += 1
                        else:
                            social_negativo += 1
                

            #salva na base de dados primarios
            dados_primarios.loc[dados_primarios['Nome do Hotel'] == hotel_key, 'TOTAL COMENTÁRIOS'] = dados_comentarios.shape[0]
            dados_primarios.loc[dados_primarios['Nome do Hotel'] == hotel_key, 'AMBIENTAL TOTAL'] = counter_ambiental
            dados_primarios.loc[dados_primarios['Nome do Hotel'] == hotel_key, 'SOCIAL TOTAL'] = counter_social
            dados_primarios.loc[dados_primarios['Nome do Hotel'] == hotel_key, 'GOVERNANÇA TOTAL'] = counter_governanca
            dados_primarios.loc[dados_primarios['Nome do Hotel'] == hotel_key, 'AMBIENTAL +'] = ambiental_postivo
            dados_primarios.loc[dados_primarios['Nome do Hotel'] == hotel_key, 'AMBIENTAL -'] = ambiental_negativo
            dados_primarios.loc[dados_primarios['Nome do Hotel'] == hotel_key, 'SOCIAL +'] = social_postivo
            dados_primarios.loc[dados_primarios['Nome do Hotel'] == hotel_key, 'SOCIAL -'] = social_negativo
            dados_primarios.loc[dados_primarios['Nome do Hotel'] == hotel_key, 'GOVERNANÇA +'] = governanca_postivo
            dados_primarios.loc[dados_primarios['Nome do Hotel'] == hotel_key, 'GOVERNANÇA -'] = governanca_negativo

    except:
        print('Erro no comentario', comentario)
        pass            

Processando hotel 1 de 196  ACasaRosa Hostel
Processando hotel 2 de 196  Apoema Suites e Flats
Processando hotel 3 de 196  Apple House
Processando hotel 4 de 196  Bahia Inn Pousada
Processando hotel 5 de 196  Casa Clara
Processando hotel 6 de 196  Centro de Férias SESC Bertioga
Processando hotel 7 de 196  Dadiva Hotel Boutique
Processando hotel 8 de 196  Estalagem Spiller
Processando hotel 9 de 196  Hotel Fazenda Bela Vista
Processando hotel 10 de 196  Hotel Fazenda Itáytyba Ecoturismo
Processando hotel 11 de 196  Hotel Fazenda Vale Amanhecer
Processando hotel 12 de 196  Hotel Paraíso do Morro
Processando hotel 13 de 196  Hotel Península
Processando hotel 14 de 196  Hotel Portal das Águas
Processando hotel 15 de 196  Hotel Pousada Kaster


In [10]:
dados_primarios.head()

,MS1,MS2,MS3,MS4,MS5,MS6,MS7,PA1,PA2,PA3,...,TOTAL COMENTÁRIOS,AMBIENTAL TOTAL,AMBIENTAL +,AMBIENTAL -,SOCIAL TOTAL,SOCIAL +,SOCIAL -,GOVERNANÇA TOTAL,GOVERNANÇA +,GOVERNANÇA -
0,7,7,7,7,7,7,7,5,7,7,...,131.0,48.0,21.0,27.0,30.0,15.0,15.0,12.0,3.0,9.0
1,7,7,7,5,7,7,7,1,7,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,7,7,7,7,7,7,4,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,7,7,7,7,7,7,7,7,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,7,7,7,7,7,7,6,6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#Exporta os dados
dados_primarios.to_excel('DadosLuisaFinalV6.xlsx', index=False)